In [3]:
# Imports
import sys
sys.path.append('../src')

import time
from datetime import date
from neuralforecast.losses.pytorch import DistributionLoss
from model.pipeline import workflow

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

2025-06-04 18:35:27.287774: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 18:35:27.374251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 18:35:27.406216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 18:35:27.415154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 18:35:27.474849: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# Parameters
stage = 'train'
start_date = '2000-01-01'
end_date = date.today()
horizon = 5
stocks = ['^GSPC', '^IXIC', '^DJI', 'GOOG', 'AAPL', 'TSLA', 'MSFT', 'NVDA', 'AMD', 'META', 'AMZN', 'NFLX', 'BTC-USD']
metrics = ['close', 'change', 'direction']

In [ ]:
start = time.time()
for stock in stocks:
    for metric_column in metrics:
        for stage in ['train', 'predict', 'forecast']:
            workflow(stage,
                     stock,
                     start_date,
                     end_date,
                     metric_column,
                     horizon=horizon)
end = time.time()
print(
    f"Total time elapsed (in minutes): "
    + "{:.2f}".format((end - start) / 60.0)
)